In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
import numpy as np
import time
from copy import deepcopy
import seaborn as sns 
import matplotlib.pyplot as plt

In [2]:
Porn=np.load("Porn_data.npy")

In [3]:
Non_Porn=np.load("Non_Porn_data.npy")

In [4]:
Porn.shape, Non_Porn.shape

((71302, 50, 50, 3), (65379, 50, 50, 3))

In [5]:
input_data=[]
for i in range(2):
    if i == 0:
        for j in range(len(Non_Porn)):
            input_data.append(Non_Porn[j])
    else :
        for j in range(len(Porn)):
            input_data.append(Porn[j])

In [6]:
input_data=np.array(input_data)

In [7]:
label=[]
for i in range(len(Non_Porn)+len(Porn)):
    if i>len(Non_Porn):
        label.append(0)
    else:
        label.append(1)

In [8]:
label=np.array(label)

In [9]:
input_data= (input_data*(1/256)-0.5)/0.5

In [11]:
input_data=torch.FloatTensor(input_data)
label=torch.LongTensor(label)

In [12]:
input_data=input_data.transpose(1,3)

In [13]:
input_data.shape

torch.Size([136681, 3, 50, 50])

In [14]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
dataset = TensorDataset(input_data,label)

In [15]:
trainset,valset=torch.utils.data.random_split(dataset,[116681,20000])
trainset,testset=torch.utils.data.random_split(trainset,[96681,20000])

partition={'train':trainset,'val':valset,'test':testset}

In [16]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1=nn.Conv2d(in_channels=3,
                            out_channels=32,
                            kernel_size=8,
                            stride=2,
                            padding=2) # 32 28 28
        self.conv2=nn.Conv2d(in_channels=32, 
                            out_channels=64,
                            kernel_size=6,
                            stride=1,
                            padding=2) # 64 60 64
        self.conv3=nn.Conv2d(in_channels=64,
                            out_channels=128,
                            kernel_size=3,
                            stride=1,
                            padding=1) # 25 30 128
        self.act=nn.ReLU()
        self.maxpool1=nn.MaxPool2d(kernel_size=2,
                                  stride=2)
        self.fc=nn.Linear(512,2)
    def forward(self,x):
        x=self.conv1(x)
        x=self.act(x)
        x=self.maxpool1(x)
        x=self.conv2(x)
        x=self.act(x)
        x=self.maxpool1(x)
        x=self.conv3(x)
        x=self.act(x)
        x=self.maxpool1(x)
        x=x.view(x.size(0),-1)
        x=self.fc(x)
        return x

In [17]:
def dimension_check():
    net=CNN()
    x=torch.randn(1,3,50,50)
    y=net(x)
    print(y.size())

In [18]:
dimension_check()

torch.Size([1, 2])


In [19]:
def train(net,partition,optimizer,criterion):
    trainloader = torch.utils.data.DataLoader(partition['train'],
                                             batch_size=4096,
                                             shuffle=True)
    net.train()
    correct = 0
    total =0
    train_loss = 0.0
    for i, data in enumerate(tqdm(trainloader, 0)):
        time.sleep(0.0000001)
        optimizer.zero_grad()
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = net(inputs)
        #break
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #if i % 200 == 0:
        #    print(correct,train_loss)
    train_loss = train_loss / len(trainloader)
    train_acc = 100 * correct / total
    return net, train_loss, train_acc

In [25]:
def test(net):
    testloader = torch.utils.data.DataLoader(partition['test'], 
                                             batch_size=4096, 
                                             shuffle=False, num_workers=2)
    net.eval()
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm(testloader):
            time.sleep(0.0000001)
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()

            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        test_acc = 100 * correct / total
    return test_acc

In [21]:
def validate(net, partition, criterion):
    valloader = torch.utils.data.DataLoader(partition['val'], 
                                            batch_size=4096, 
                                            shuffle=False, num_workers=2)
    net.eval()

    correct = 0
    total = 0
    val_loss = 0 
    with torch.no_grad():
        for data in tqdm(valloader):
            time.sleep(0.0000001)
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = net(images)

            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(valloader)
        val_acc = 100 * correct / total
    return val_loss, val_acc

In [22]:
from tqdm import tqdm, tqdm_notebook
from tqdm import trange
import time

In [23]:
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
net=CNN()
net.cuda()
criterion=nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0015, weight_decay=0.00001)
train_losses=[]
val_losses=[]
train_accs=[]
val_accs=[]
for epoch in range(100):
    ts=time.time()
    net, train_loss,train_acc=train(net, partition, optimizer, criterion)
    val_loss, val_acc = validate(net, partition, criterion)
    te=time.time()
    print('time : {:2.2f}, train_loss : {:2.2f}, train_acc : {:2.2f},val_acc : {:2.2f},val_loss : {:2.2f}'.format(te-ts,train_loss,train_acc,val_acc,val_loss))
    train_loss=round(train_loss,2)
    val_loss=round(val_loss,2)
    train_acc=round(train_acc,2)
    val_acc=round(val_acc,2)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)


  4%|▍         | 1/24 [00:00<00:03,  6.47it/s]

time : 4.16, train_loss : 0.48, train_acc : 78.21,val_acc : 82.06,val_loss : 0.43


  4%|▍         | 1/24 [00:00<00:03,  5.86it/s]

time : 3.87, train_loss : 0.40, train_acc : 83.07,val_acc : 83.77,val_loss : 0.38


  4%|▍         | 1/24 [00:00<00:03,  6.95it/s]

...

  4%|▍         | 1/24 [00:00<00:03,  6.82it/s]

time : 3.89, train_loss : 0.14, train_acc : 94.56,val_acc : 92.56,val_loss : 0.20


  0%|          | 0/5 [00:00<?, ?it/s]

time : 4.08, train_loss : 0.14, train_acc : 94.86,val_acc : 93.36,val_loss : 0.18


100%|██████████| 5/5 [00:00<00:00,  9.83it/s]


In [26]:
test_acc = test(net)
test_acc=round(test_acc,2)

100%|██████████| 5/5 [00:00<00:00,  9.89it/s]


In [27]:
test_acc

93.11

In [28]:
np.save("basic_train_accs",train_accs)
np.save("basic_val_accs",val_accs)
np.save("basic_train_losses",train_losses)
np.save("basic_val_losses",train_losses)